In [1]:
import torch
from torchvision import transforms
import time
import  numpy as np
import logging
import os
import gc
from matplotlib import pyplot as plt
import sys
import json

import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from utils import EarlyStopping, LRScheduler
from models.Captioner import Encoder, DecoderWithAttention, Decoder_NoTFNoAttNoGlove
from data import CaptionDataset
from embedding import get_embedding_matrix


gc.collect()
torch.cuda.empty_cache()

# Data parameters
train_file = '/home/gaurangajitk/DL/data/image-caption-data/annotations_train1.txt'
val_file = '/home/gaurangajitk/DL/data/image-caption-data/annotations_val.txt'
data_folder = '/home/gaurangajitk/DL/data/image-caption-data'  # folder with data files saved by create_input_files.py
data_name = '5_cap_per_img_15_min_word_freq'  # base name shared by data files

In [52]:
import torch
from torch.utils.data import Dataset
import h5py
import json
import os
import numpy as np

class CaptionDataset(Dataset):
    """
    A PyTorch Dataset class to be used in a PyTorch DataLoader to create batches.
    """

    def __init__(self, data_folder, data_name, split, transform=None):
        """
        :param data_folder: folder where data files are stored
        :param data_name: base name of processed datasets
        :param split: split, one of 'TRAIN', 'VAL', or 'TEST'
        :param transform: image transform pipeline
        """
        super().__init__()
        self.split = split
        assert self.split in {'TRAIN', 'VAL', 'TEST'}

        # Open hdf5 file where images are stored
        self.h = h5py.File(os.path.join(data_folder, self.split + '_IMAGES_' + data_name + '.hdf5'), 'r')
        self.imgs = self.h['images']

        # Captions per image
        self.cpi = self.h.attrs['captions_per_image']

        # Load encoded captions (completely into memory)
        with open(os.path.join(data_folder, self.split + '_CAPTIONS_' + data_name + '.json'), 'r') as j:
            self.captions = json.load(j)

        # Load caption lengths (completely into memory)
        with open(os.path.join(data_folder, self.split + '_CAPLENS_' + data_name + '.json'), 'r') as j:
            self.caplens = json.load(j)
        
        # Load meta image IDs (completely into memory)
        with open(os.path.join(data_folder, self.split + '_META_IMAGES_' + data_name + '.json'), 'r') as j:
            self.meta_im = json.load(j)
        
        # if split == 'VAL':
        #     self.imgs = self.imgs[1000:1500]
        #     self.captions = self.captions[1000:1500]
        #     self.caplens = self.caplens[1000:1500]

        # PyTorch transformation pipeline for the image (normalizing, etc.)
        self.transform = transform

        # Total number of datapoints
        self.dataset_size = len(self.captions)

    def __getitem__(self, i):
        # Remember, the Nth caption corresponds to the (N // captions_per_image)th image
        img = torch.FloatTensor(self.imgs[i // self.cpi] / 255.)

        if self.transform is not None:
            img = self.transform(img)

        caption = torch.LongTensor(self.captions[i])

        caplen = torch.LongTensor([self.caplens[i]])

        if self.split == 'TRAIN':
            return img, caption, caplen
        else:
            # For validation of testing, also return all 'captions_per_image' captions to find BLEU-4 score
            all_captions = torch.LongTensor(
                self.captions[((i // self.cpi) * self.cpi):(((i // self.cpi) * self.cpi) + self.cpi)])
            return img, caption, caplen, all_captions, self.meta_im[i // self.cpi]

    def __len__(self):
        return self.dataset_size

In [57]:
word_map_file = os.path.join(data_folder, 'WORDMAP_' + data_name + '.json')
with open(word_map_file, 'r') as j:
    word_map = json.load(j)

config = {
        'encoded_image_size': 14,
        'fine_tune_encoder': False,
        'attention_dim': 512,
        'embed_dim': 200,
        'decoder_dim': 512,
        'dropout': 0.5,
        'encoder_dim': 2048,
        'encoder_lr': 1e-4,
        'decoder_lr': 1e-3,
        'batch_size': 10
    }

print('Preparing Datasets')
data_transforms = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),    #add random flip transform later
        ])

trainset = CaptionDataset(data_folder, data_name, 'TRAIN', transform=data_transforms)
valset = CaptionDataset(data_folder, data_name, 'VAL', transform=data_transforms)
testset = CaptionDataset(data_folder, data_name, 'TEST', transform=data_transforms)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'], shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(valset, batch_size=config['batch_size'], shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(testset, batch_size=config['batch_size'], shuffle=True, num_workers=1)

Preparing Datasets


In [59]:
valset[9][-1]

'VizWiz_val_00000001.jpg'

In [60]:
val_batch = next(iter(val_loader))

In [61]:
val_batch[-1]

('VizWiz_val_00000888.jpg',
 'VizWiz_val_00000406.jpg',
 'VizWiz_val_00001111.jpg',
 'VizWiz_val_00000292.jpg',
 'VizWiz_val_00000080.jpg',
 'VizWiz_val_00001121.jpg',
 'VizWiz_val_00001269.jpg',
 'VizWiz_val_00000787.jpg',
 'VizWiz_val_00000550.jpg',
 'VizWiz_val_00000370.jpg')

In [3]:
print('Creare Model')
encoder = Encoder(config)
decoder = dec(config, vocab_size=len(word_map))

Creare Model


In [ ]:
print('Load pretrained glove embeddidngs')
word_embedding = get_embedding_matrix(word_map.keys())
decoder.load_pretrained_embeddings(word_embedding)
decoder.fine_tune_embeddings()

Load pretrained glove embeddidngs
Pre-processing /home/gaurangajitk/DL/data/image-caption-data/glove.6B.200d.txt vectors...
2821 / 2849 tokens have corresponding embedding vector


In [11]:
# train_batch = iter(train_loader)
val_batch = next(iter(val_loader))

/tmp/ipykernel_1739/3985849334.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  meta_cap = np.array(self.meta_cap[i // self.cpi]).reshape(1, -1)


1500 (1, 5) ['VizWiz_val_00000950.jpg'] [[list(['a', 'black', 'laptop', 'computer', 'showing', 'a', 'page', 'of', 'computer', 'organization', 'and', 'design'])
  list(['blue', 'book', 'cover', 'of', 'hardware', 'and', 'software', 'textbook'])
  list(['a', 'pdf', 'of', 'a', 'book', 'on', 'a', 'computer', 'screen', 'about', 'computer', 'organization', 'and', 'design'])
  list(['an', 'image', 'of', 'a', 'book', 'on', 'computer', 'organization', 'and', 'design', 'with', 'its', 'front', 'and', 'back', 'cover', 'facing', 'out', 'is', 'presented', 'on', 'the', 'screen', 'of', 'a', 'computer'])
  list(['a', 'browser', 'window', 'shows', 'a', 'book', 'cover', 'that', 'is', 'a', 'scanned', 'image'])]]
1500 (1, 5) ['VizWiz_val_00000105.jpg'] [[list(['a', 'screenshot', 'of', 'a', 'computer', 'monitor', 'showing', 'an', 'error', 'for', 'no', 'device', 'drivers', 'being', 'found'])
  list(['a', 'windows', 'computer', 'message', 'that', 'says', 'no', 'device', 'drivers', 'were', 'found'])
  list(['a'

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 62, in default_collate
   

In [5]:
val_batch[-1]

[('VizWiz_val_00000663.jpg',)]

In [5]:
image, captions, cap_len = train_batch
encoder_out = encoder(image)

In [6]:
predictions, encoded_captions, decode_lengths, alphas, sort_ind = decoder(encoder_out, captions, cap_len)

In [7]:
predictions.shape

torch.Size([2, 13, 2849])

In [ ]:
a = pack_padded_sequence(predictions, decode_lengths, batch_first=True)
targets = pack_padded_sequence(encoded_captions, decode_lengths, batch_first=True)

In [ ]:
print(a.data.shape, targets.data.shape)

torch.Size([423, 2849]) torch.Size([423])


In [ ]:
_, ind = a.data.topk(2, 1, True, True)

In [ ]:
correct = targets.data.view(-1, 1).expand_as(ind).eq(targets.data.view(-1, 1).expand_as(ind))

In [ ]:
correct.view(-1).float().sum()

tensor(846.)

In [ ]:
batch_size = encoder_out.size(0)
encoder_dim = encoder_out.size(-1)  #2048
vocab_size = decoder.vocab_size

# Flatten image
encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
num_pixels = encoder_out.size(1)

# Sort input data by decreasing lengths; why? apparent below
caption_lengths, sort_ind = cap_len.squeeze(1).sort(dim=0, descending=True)
encoder_out = encoder_out[sort_ind]
encoded_captions = captions[sort_ind]

# Embedding
embeddings = decoder.embedding(encoded_captions)  # (batch_size, max_caption_length, embed_dim)

# Initialize LSTM state
h, c = decoder.init_hidden_state(encoder_out)  # (batch_size, decoder_dim)

# We won't decode at the <end> position, since we've finished generating as soon as we generate <end>
# So, decoding lengths are actual lengths - 1
decode_lengths = (caption_lengths - 1).tolist()

# Create tensors to hold word predicion scores and alphas
predictions = torch.zeros(batch_size, max(decode_lengths), vocab_size)
alphas = torch.zeros(batch_size, max(decode_lengths), num_pixels)

# At each time-step, decode by
# attention-weighing the encoder's output based on the decoder's previous hidden state output
# then generate a new word in the decoder with the previous word and the attention weighted encoding
for t in range(min(1, max(decode_lengths))):
    batch_size_t = sum([l > t for l in decode_lengths])
    attention_weighted_encoding, alpha = decoder.attention(encoder_out[:batch_size_t],
                                                        h[:batch_size_t])
    gate = decoder.sigmoid(decoder.f_beta(h[:batch_size_t]))  # gating scalar, (batch_size_t, encoder_dim)
    attention_weighted_encoding = gate * attention_weighted_encoding
    h, c = decoder.decode_step(
        torch.cat([embeddings[:batch_size_t, t, :], attention_weighted_encoding], dim=1),
        (h[:batch_size_t], c[:batch_size_t]))  # (batch_size_t, decoder_dim)
    preds = decoder.fc(decoder.dropout(h))  # (batch_size_t, vocab_size)
    predictions[:batch_size_t, t, :] = preds
    alphas[:batch_size_t, t, :] = alpha

In [ ]:
predictions.shape

torch.Size([32, 20, 2849])

In [ ]:
imgs, caps, caplens, allcaps = val_batch
allcaps.shape

torch.Size([32, 5, 102])

In [ ]:
allcaps = allcaps[sort_ind]
references = list()

In [ ]:
for j in range(allcaps.shape[0]):
    img_caps = allcaps[j].tolist()
    img_captions = list(
        map(lambda c: [w for w in c if w not in {word_map['<start>'], word_map['<pad>']}],
            img_caps))  # remove <start> and pads
    references.append(img_captions)               

In [ ]:
len(references[0][2])

18

In [ ]:
_, preds = torch.max(predictions, dim=2)

In [ ]:
preds = preds.tolist()
temp_preds = list()
for j, p in enumerate(preds):
    temp_preds.append(preds[j][:decode_lengths[j]])  # remove pads
preds = temp_preds

In [ ]:
for i, batch in enumerate(val_loader):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/gaurangajitk/anaconda3/envs/cascades/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/gaurangajitk/DL/image-captioning/data.py", line 58, in __getitem__
    all_captions = torch.LongTensor(
ValueError: expected sequence of length 157 at dim 1 (got 102)


In [ ]:
with open(os.path.join(data_folder, 'VAL' + '_CAPTIONS_' + data_name + '.json'), 'r') as j:
    captions = json.load(j)

In [ ]:
for i in range(4429,len(captions)):
    assert len(captions[i]) == 102, f'{i}'

AssertionError: 4430

In [ ]:
valset.imgs[4430].shape

(3, 256, 256)

In [ ]:
plt.imshow(valset[4430][0].transpose(1,2,0))

ValueError: expected sequence of length 157 at dim 1 (got 102)

In [ ]:
len(valset.caplens)

38250